In [ ]:
""" 
Make plots of drawings, with various goals, generally summarizing over dataset.
e.g.,:
- Primitives, plotting change over trials (conditiioned on a single primtive)
TODO:
Move this to text. is very slow plotting here.

UPDATE:
7/27/21 - Couple things extensively written:
(1) plotting each unique prmitive, day x rendition
(2) Plotting "primitiveness" comparing motor behavior within and across cateogires.

TODOs:
See within.
"""


### Plot all primtiives, randomly picked out over time (takes long time)

In [ ]:

# %cd ..
# from tools.utils import * 
# from tools.plots import *
# from tools.analy import *
# from tools.calc import *
# from tools.analyplot import *
# from tools.preprocess import *
# from tools.dayanalysis import *

# from pythonlib.drawmodel.analysis import *
# from pythonlib.tools.stroketools import *
from pythonlib.dataset.dataset import Dataset
# import pickle
# import os
import numpy as np
import matplotlib.pyplot as plt

#     a = "Diego"
e = "primitivesall"
r = "null"

for a in ["Diego", "Pancho"]:

    D = Dataset([])
    D.load_dataset_helper(a, e, rule=r)

    # Extract pre-saved tasks
    D.load_tasks_helper()

    SAVEDIR = f"/data2/analyses/main/primitives/{a}-{e}-{r}/FIGS/drawfig"
    from pythonlib.tools.expttools import writeDictToYaml, makeTimeStamp
    notes = {
        "tstamp":makeTimeStamp(),
        "code":"dataset/analy_dataset_drawfigplots_210623",
        "goal":"visualize progression during learning"
    }
    import os 
    os.makedirs(SAVEDIR, exist_ok=True)
    writeDictToYaml(notes, f"{SAVEDIR}/notes.yaml")

#     ##### Pull out trials based on task filters
#     primitives_list = [[dd[0] for dd in d.Shapes] for d in D.Dat["Task"].tolist()]

    
    ##### Plot, randomly interleaved trials over all time.
    # 1) Find all trials for a given primitive
    prim_list = D.Dat["character"].unique()
    # prim_list
    for PRIM in prim_list:
    #     PRIM = "_square"

        for CENTERIZE in [False, True]:
    #         CENTERIZE = True

            # D.Dat["task_stagecategory"]==PRIM
            # D.filterPandas
            SAVEDIR_THIS = f"{SAVEDIR}/{PRIM}-centerize_{CENTERIZE}"
            os.makedirs(SAVEDIR_THIS, exist_ok=True)

            Dprim = D.filterPandas({"character":[PRIM]}, "dataset")
            tvals = Dprim.Dat["tval"].tolist()
            assert all(np.diff(tvals)>0), "need to sort..."

            # Print all the variations in tasks:
            sx_list = []
            sy_list = []
            x_list = []
            y_list = []
            theta_list = []
            for T in Dprim.Dat["Task"].tolist():
                sx = T.Shapes[0][1]["sx"]
                sy = T.Shapes[0][1]["sy"]
                sx_list.append(sx)
                sy_list.append(sy)

                xpos = T.Shapes[0][1]["x"]
                x_list.append(xpos)
                y_list.append(T.Shapes[0][1]["y"])

                theta_list.append(T.Shapes[0][1]["theta"])

            import matplotlib.pyplot as plt
            fig, axes = plt.subplots(1,3, figsize=(15,3))

            ax = axes.flatten()[0]
            ax.plot(sx_list, sy_list, "xk", alpha=0.2)
            ax.set_xlabel("sx")
            ax.set_ylabel("sy")
            ax = axes.flatten()[1]
            ax.plot(x_list, y_list, "xk", alpha=0.2)
            ax.set_xlabel("x")
            ax.set_ylabel("y")
            ax = axes.flatten()[2]
            ax.plot(theta_list, np.ones_like(theta_list), "xk", alpha=0.2)
            ax.set_xlabel("theta")
            # ax.set_ylabel("y")
            fig.savefig(f"{SAVEDIR_THIS}/feature_distrubtions.pdf")



            import numpy as np

            min_trials_allow_plot = 4 # min num trial
            ignore_position=True
            Nplot = 64

            ##### Pull out each unique combo of (position, scale)
            from itertools import product

            if ignore_position:
                vals_list = product(np.unique(sx_list), np.unique(sy_list), np.unique(theta_list))
            else:
                vals_list = product(np.unique(sx_list), np.unique(sy_list), np.unique(x_list), np.unique(y_list), np.unique(theta_list))

            for vals in vals_list:

                # Extract trials matching these values
                if ignore_position:
                    sx, sy, th = vals
                    F = {"sx":[sx], "sy":[sy], "theta":[th]}
                    fname = f"sx{sx}_sy{sy}_th{th}"
                else:
                    sx, sy, x, y, th = vals
                    F = {"sx":[vals[0]], "sy":[vals[1]], "x":[vals[2]], "y":[vals[3]], "theta":[vals[4]]}
                    fname = f"x{x}_y{y}_sx{sx}_sy{sy}_th{th}"

                Dprimsub = Dprim.filterByTask(F)
                if len(Dprimsub.Dat)==0:
                    continue
                if len(Dprimsub.Dat)<min_trials_allow_plot:
                    continue

                # === get uniformly sampled trials over all time.
                ninds = len(Dprimsub.Dat)
                N = Nplot
                N = min(ninds, N)
                indices = list(np.floor(np.linspace(0, ninds-1, N)))
                indices = [int(i) for i in indices]
                trialcodes = Dprimsub.Dat.iloc[indices]["trialcode"].tolist()

                fig1 = Dprimsub.plotMultTrials(indices, ncols=8, centerize=CENTERIZE, titles=trialcodes);
                fig1.suptitle(f"N={ninds}")
                fig2 = Dprimsub.plotMultTrials(indices, which_strokes="strokes_task", ncols=8, centerize=CENTERIZE, 
                                               titles=trialcodes);

                fig1.savefig(f"{SAVEDIR_THIS}/{fname}-BEH.pdf")
                fig2.savefig(f"{SAVEDIR_THIS}/{fname}-TASK.pdf")


### More careful, plot primtiive changes over time

In [ ]:
from pythonlib.dataset.dataset_analy.primitives import *

from pythonlib.dataset.dataset import Dataset
# import pickle
# import os
import numpy as np
import matplotlib.pyplot as plt

PLOT_RAW = True
only_keep_random_tasks = True # True, since in general for prims I made them random.

#     a = "Diego"
for a in ["Pancho", "Diego"]:
    e = "primcat12"
    r = "null"
    # a = "Pancho"

    D = Dataset([])
    D.load_dataset_helper(a, e, rule=r)

    # Extract pre-saved tasks
    D.load_tasks_helper()


    SAVEDIR = f"/data2/analyses/main/primitives/{a}-{e}-{r}/FIGS/drawfig_by_date"
    from pythonlib.tools.expttools import writeDictToYaml, makeTimeStamp
    notes = {
        "tstamp":makeTimeStamp(),
        "code":"dataset/analy_primitives_081721",
        "goal":"visualize progression during learning"
    }

    import os 
    os.makedirs(SAVEDIR, exist_ok=True)
    writeDictToYaml(notes, f"{SAVEDIR}/notes.yaml")



    # === Preprocess
    D = preprocess(D)

    # ==== Define that dataset to plot.
    for TRAIN in ["train", "test"]:
        Dthis = D.filterPandas({"random_task":[only_keep_random_tasks], "monkey_train_or_test":[TRAIN]}, "dataset")

        SAVEDIR_THIS = f"{SAVEDIR}/{TRAIN}"

        # === Plot date vs. trial
        if PLOT_RAW:
            # === Get primlist
            prim_list = extract_primlist(Dthis)
            print(prim_list)
            plot_beh_day_vs_trial(Dthis, prim_list,SAVEDIR_THIS)
        
        # Compute "primitiveness" based on motor consistency
        plot_all(Dthis, SAVEDIR_THIS, a, e)


##### [More recent than above] Plot combined primitives, but NICE (primitivs x trial), separate plots for early/late x train/test

In [144]:
from pythonlib.dataset.dataset_analy.primitives import *
from pythonlib.dataset.dataset import Dataset
import numpy as np
import matplotlib.pyplot as plt
import os
from pythonlib.dataset.dataset_analy.primitives import plot_beh_all_combined


PLOT_BEH = False # primiitves x trial.
PLOT_MOTOR_STATS = True
#     a = "Diego"
ORIENTATION = "horizontal"
NITER = 5
for a in ["Pancho", "Diego"]:
    e = "primcat12"
    r = "null"

    D = Dataset([])
    D.load_dataset_helper(a, e, rule=r)
    D.load_tasks_helper()

    SAVEDIR = f"/data2/analyses/main/primitives/{a}-{e}-{r}/FIGS/drawfig_by_date"
    os.makedirs(SAVEDIR, exist_ok=True)

    # === Preprocess
    D = preprocess(D, a, e)

    
    if PLOT_BEH:
        assert False, "too slow - move to script, suppress plots"

        # ==== Define that dataset to plot.
        plot_beh_all_combined(D, a, e, SAVEDIR, ORIENTATION, NITER)

        plt.close("all")
                    
    if PLOT_MOTOR_STATS:
        Nplot = 20
        for TRAIN_TEST in ["train", "test"]:

            for EPOCH in ["early", "late"]:

                ### preprocess
                Dthis, DictDateEarlyLate, DictDay, DictDateInds = clean_and_group_data(D, TRAIN_TEST, a, e, EPOCH, SAVEDIR_THIS=None)

                #### Improving quantification:
                # 1. Motor consistency
                # 2. Difference between early and late, trained vs. untrained

                # Plot behavior, showing velocities.
                list_primtuple = Dthis.Dat["primtuple"].tolist()
                for j, primthis in enumerate(list_primtuple):
        #             if True:
        #                 primthis = "['_zigzag-random', array(1.7), array(1.7), array(1.57079633)]"
        #             else:
        #                 primthis = list_primtuple[9]
                    # SAVEDIR
                    SAVEDIR_THIS = f"/data2/analyses/main/primitives/{a}-{e}-{r}/FIGS/QUANTIFICATION_MOTOR/{TRAIN_TEST}/{primthis}"
                    os.makedirs(SAVEDIR_THIS, exist_ok=True)

                    list_inds = Dthis.Dat[Dthis.Dat["primtuple"]==primthis].index.tolist()

                    if len(list_inds)<5:
                        continue

                    figbeh, idxs = Dthis.plotMultTrials(list_inds, color_by="speed", nrand=Nplot, return_idxs=True);
                    if idxs is None:
                        continue
                    figtask = Dthis.plotMultTrials(idxs, which_strokes="strokes_task");

                    #### Motor stats (including onset, duration etc).
                    # Get time from go --> first touch (onset of stroke 1)
                    # - Sanity check.
                    figspeed1 = Dthis.plotMultTrialsTimecourse(idxs, "speed", align_to="go_cue");
                    figspeed2 = Dthis.plotMultTrialsTimecourse(idxs, "speed", align_to="first_stroke_onset");

                    # Plot all summary motor stats (just for the plotted trials)
                    stats_names = Dthis.get_motor_stats(0).keys()
                    ncols = 3
                    nrows = int(np.ceil(len(stats_names)/ncols))
                    fig, axes = plt.subplots(nrows, ncols, sharex=True, figsize=(2*nrows, 3.5*ncols))

                    for ax, name in zip(axes.flatten(), stats_names):
                        times = [Dthis.get_motor_stats(i)[name] for i in idxs]
                        ax.plot(times, '-ok')
                        ax.set_title(name)
                        ax.set_ylim(bottom=0.)

                    # SAVE FIGS
                    figbeh.savefig(f"{SAVEDIR_THIS}/beh-speed-{EPOCH}.pdf")
                    figtask.savefig(f"{SAVEDIR_THIS}/task-speed-{EPOCH}.pdf")
                    figspeed1.savefig(f"{SAVEDIR_THIS}/timecourse-speed-{EPOCH}-1.pdf")
                    figspeed2.savefig(f"{SAVEDIR_THIS}/timecourse-speed-{EPOCH}-2.pdf")
                    fig.savefig(f"{SAVEDIR_THIS}/motor_stats-{EPOCH}.pdf")
                    plt.close("all")
    
    
                    ## TODO: plandir plots - entire distribution of motor stats.



Searching using this string:
/data2/analyses/database//*Pancho*primcat12*null*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
0
Searching using this string:
/data2/analyses/database/BEH/*Pancho*primcat12*null*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
1
---
/data2/analyses/database/BEH/Pancho-primcat12-null-210907_224122
----------------
Currently loading: /data2/analyses/database/BEH/Pancho-primcat12-null-210907_224122
Loaded metadat:
{'sketchpad_edges': array([[-311.84, -224.8 ],
       [ 311.84,  429.6 ]]), 'metadat_probedat': {'sdate': 210731, 'edate': 210805, 'strokmodel_kind': None, 'strokmodel_tstamp': None, 'datecategories': {'210731': 1, '210801': 1, '210802': 1, '210803': 1, '210804': 1, '210805': 1}, 'dates_for_summary': [], 'matchedstrokes': None, 'exptnames': ['primcat1', 'primcat2'], 'T1': [], 'G1': [], 'G2': [], 'G3': [], 'G4': [], 'description': '', 'finalized': True, 'good_expt': True, 'expt': 'primcat12', 'task_train_test': {'p

TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check 

TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check 

TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check 

TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check out of bounds
TODO: check 

...

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-144-0741ee849977>", line 66, in <module>
    figbeh, idxs = Dthis.plotMultTrials(list_inds, color_by="speed", nrand=Nplot, return_idxs=True);
  File "/data1/code/python/pythonlib/pythonlib/dataset/dataset.py", line 3670, in plotMultTrials
    fig= plotGridWrapper(data, plotfunc, ncols=ncols, titles=titles,naked_axes=naked_axes, origin="top_left")
  File "/data1/code/python/pythonlib/pythonlib/tools/plottools.py", line 590, in plotGridWrapper
    plotfunc(dat, ax)
  File "/data1/code/python/pythonlib/pythonlib/dataset/dataset.py", line 3620, in <lambda>
    plotfunc = lambda data, ax: plotDatStrokesMapColor(
  File "/data1/code/python/pythonlib/pythonlib/drawmodel/strokePlots.py", line 109, in plotDatStrokesMapColor
    sc = ax.scatter(s[:,0], s[:,1

TypeError: object of type 'NoneType' has no len()

Error in callback <function flush_figures at 0x7fc6182a7040> (for post_execute):


KeyboardInterrupt: 

#### (General purpose) preprocess dataset to expose motor stats

##### Behavior distance metric based on comparing velocity timecourses

In [ ]:
# NOT SURE IF NEED THIS ANYMORE
def _hash(primtuple):
    return "_".join([str(p) for p in primtuple])

# For each primtuple, check if has enough trials early and late
# Output is, for each primtuple, all the inds present on each date.
DictDateInds = {}
for primtuple in list_primtuple:
#     inds = D.filterPandas({"character":[primtuple[0]], "sx":[primtuple[1]], 
#                                     "sy":[primtuple[2]], "theta":[primtuple[3]]})
#     list_days = D.Dat.iloc[inds]["date"]
    
    DictDateInds[_hash(primtuple)] = {}
    print(primtuple)
    for day in list_days:
        # save the indices and dates for this primtuple
        inds = DthisCentered.filterPandas({"date":[day], "character":[primtuple[0]], "sx":[primtuple[1]], 
                                        "sy":[primtuple[2]], "theta":[primtuple[3]]})
        
        DictDateInds[_hash(primtuple)][str(day)] = inds
        

In [ ]:
#### GET DISTANCES
from pythonlib.drawmodel.strokedists import distscalarStrokes, distanceStroksMustBePaired

def dfunc(strokes1, strokes2):
    def _pick_single_strok(strokes):
        # If strokes is len>1, then returns the longest strok
        # takes most pts as proxy (not realy longest)
        if len(strokes)==1:
            return strokes
        list_lens = [s.shape[0] for s in strokes]
        ind = np.argmax(list_lens)
        return [strokes[ind]]
        
    # if any are >1 strokes, just take the first
    strokes1 = _pick_single_strok(strokes1)
    strokes2 = _pick_single_strok(strokes2)

#     return distanceStroksMustBePaired(strokes1, strokes2, ver="euclidian")
    return distanceStroksMustBePaired(strokes1, strokes2, ver="euclidian_diffs")


# For each prituple and day, compute a within-day score
import random

nmin = 6
max_comparisons = 50

OUT = []
for primtuple in list_primtuple:
    for day in list_days:
        inds = DictDateInds[_hash(primtuple)][day]
        if len(inds)<nmin:
            continue
        
        # Get pairwise between all inds
        random.shuffle(inds)
        ct = 0
        print(primtuple, day)
        while ct<max_comparisons:
            for i1 in inds:
                for i2 in inds:
                    if i1!=i2:
                        strokes1 = DthisCentered.Dat.iloc[i1]["strokes_beh"]
                        strokes2 = DthisCentered.Dat.iloc[i2]["strokes_beh"]
                        ct+=1
                        
                        dist = dfunc(strokes1, strokes2)
                        
                        OUT.append({
                            "primtuple":_hash(primtuple),
                            "prim":primtuple[0],
                            "sx":primtuple[1],
                            "sy":primtuple[2],
                            "th":primtuple[3],
                            "dist":dist,
                            "inds":(i1, i2),
                            "date":day
                        })


# Convert to df for analysis.
import pandas as pd
DF = pd.DataFrame(OUT)

# Aggregate 

# from pythonlib.tools.pandastools import 

# convert dates to ints
def _F(x):
    return int(x["date"])

DF = applyFunctionToAllRows(DF, _F, "date")

##### optional

In [ ]:
#### GET DISTANCES
# For each prituple and day, compute a within-day score
# [OPTIONAL - TAKES LOT OF TIME. Do this if want day to day qwauntificaiton. not just first and last]
import random

nmin = 6
max_comparisons = 50

OUT = []
for primtuple in list_primtuple:
    for day in list_days:
        inds = DictDateInds[_hash(primtuple)][day]
        if len(inds)<nmin:
            continue
        
        # Get pairwise between all inds
        random.shuffle(inds)
        ct = 0
        print(primtuple, day)
        while ct<max_comparisons:
            for i1 in inds:
                for i2 in inds:
                    if i1!=i2:
                        strokes1 = DthisCentered.Dat.iloc[i1]["strokes_beh"]
                        strokes2 = DthisCentered.Dat.iloc[i2]["strokes_beh"]
                        ct+=1
                        
                        dist = dfunc(strokes1, strokes2)
                        
                        OUT.append({
                            "primtuple":_hash(primtuple),
                            "prim":primtuple[0],
                            "sx":primtuple[1],
                            "sy":primtuple[2],
                            "th":primtuple[3],
                            "dist":dist,
                            "inds":(i1, i2),
                            "date":day
                        })


# Convert to df for analysis.
import pandas as pd
DF = pd.DataFrame(OUT)

# Aggregate 

# from pythonlib.tools.pandastools import 

# convert dates to ints
def _F(x):
    return int(x["date"])

DF = applyFunctionToAllRows(DF, _F, "date")

# Plot, for each primitive, distance over days
import seaborn as sns
fig = sns.catplot(data=DF, x="date", y="dist", col="primtuple", col_wrap=3, kind="point")
from pythonlib.tools.snstools import rotateLabel
rotateLabel(fig)

fig.savefig(f"{SDIRTHIS}/withinprimdist_eachday_eachprim.pdf")

In [ ]:
# 1) score within-category vs. across category

# 1b) same, but for visual similarity only


# 2) Due to motor variation? or strategy change



In [ ]:
# Pick out good beginning and ending phases for looking at primitives

# (chunk each day into first and seconds halves)

# for each chunk, compute primitiveness



In [ ]:
## TODO:
# 1) Only plot at end of day (or at least not during warmup)
# 2) plot aligned to timecoures over expt
# 3) Plot summary score over time. 

# - scoring "primitiveness"
# - score motor program consistency
# 	- even with variation across location and scale
# see other notes (Behavior spreadsheet)

### SCRATCH

In [ ]:
D.Dat["Task"].tolist()[-101].Shapes

##### general purpose, filter based on task program

In [ ]:
ObjectsAll = []
for i in range(len(D.Dat)):
    T = D.Dat.iloc[i]["Task"]

    Tml2 = T.Params["input_params"]
    Tml2.objects_extract()
    ObjectsAll.append(Tml2.Objects)
    


In [ ]:
import matplotlib.pyplot as plt

x_nums = [len(O) for O in ObjectsAll]
plt.figure()
plt.hist(x_nums)

for O in ObjectsAll:
    print([oo["obj"] for oo in O])


In [ ]:
taskobj = D.Dat.iloc[0]["Task"].Params["input_params"]

from pythonlib.drawmodel.taskgeneral import TaskClass
T = TaskClass()
T.initialize("ml2", taskobj)
T.shapes2strokes(T.Shapes)

In [ ]:
##### Expose in pandas the task features
from pythonlib.tools.pandastools import applyFunctionToAllRows
def F(x):
    objlist = x["Task"].Shapes
    if "circle" in [o["obj"] for o in objlist]:
        return True
    else:
        return False
    
D.Dat = applyFunctionToAllRows(D.Dat, F, "test")

In [ ]:
D.Dat.iloc[0]["Task"].Shapes

##### Plot

In [ ]:
import seaborn as sns
sns.scatterplot(data=Dprim.Dat, x="tval", y="score_final")

## SCRATCH

In [ ]:
## Troubleshooting assignment of names of objects/shapes.
if False:
    T = D.Dat.iloc[164]["Task"]
    # print(T.Shapes)
    T.Params["input_params"].objects_extract(fail_if_no_match=False)
    # T.Params["input_params"].Objects
    Tml2 = T.Params["input_params"]

    Tml2.objects_extract(fail_if_no_match=False)
    Tml2.Objects

    for i in range(len(D.Dat)):
        if D.Dat.iloc[i]["Task"].Shapes[0][1] is None:
            print(i)
